In [1]:
import torch
import numpy as np

In [2]:
class HMM:
    def __init__(self, K, dtype=torch.double):
        self.K = K
        self.dtype = dtype
        self._initial_prob()
        self._transition_prob()
    
    def _initial_prob(self):
        pi = torch.rand(self.K, dtype=self.dtype)
        self.initial_prob = pi / pi.sum()
    
    def _transition_prob(self):
        A = torch.rand(self.K, self.K, dtype=self.dtype)
        self.transition_prob = A / A.sum(dim=1, keepdims=True)
    
    def fit(self, X, iters=10):
        for i in range(iters):
            gamma, xi, c_list = self.expect(X)
            self.maximize(X, gamma, xi)
            ll = torch.log(c_list).sum()
        
    def expect(self, X):
        pxz_list = torch.exp(self.log_pxz(X))
        
        alpha0 = self.initial_prob * pxz_list[0]
        alpha_list = [alpha0 / alpha0.sum()]
        c_list = [alpha0.sum()]
        
        for pxz in pxz_list[1:]:
            alpha = torch.matmul(alpha_list[-1], self.transition_prob) * pxz
            alpha_list.append(alpha/alpha.sum())
            c_list.append(alpha.sum())
        
        beta_list = [torch.ones(self.K, dtype=self.dtype)]
        for i in range(len(pxz_list)-1, 0, -1):
            c = c_list[i]
            pxz = pxz_list[i]
            beta = torch.matmul(self.transition_prob, pxz * beta_list[-1]) / c
            beta_list.append(beta)
        beta_list = beta_list[::-1]
        
        alpha_list = torch.stack(alpha_list)
        c_list = torch.Tensor(c_list)
        beta_list = torch.stack(beta_list)
        
        gamma = alpha_list * beta_list
        xi = self.transition_prob * pxz_list[1:, None, :] * \
            beta_list[1:, None, :] * alpha_list[:-1, :, None] / c_list[1:, None, None]
        return gamma, xi, c_list

In [3]:
class GaussianHMM(HMM):
    def __init__(self, K, ndim, means=None, covs=None):
        super(GaussianHMM, self).__init__(K)
        self.means = means
        self.covs = covs
        
        if means is None:
            self.means = torch.zeros(K, ndim, dtype=self.dtype)
        if covs is None:
            self.covs = torch.stack([torch.eye(ndim)]*K).to(self.dtype)
        
    def log_pxz(self, X):
        N, D = X.shape
        covs_chol = torch.linalg.cholesky(self.covs)
        precision_col = torch.inverse(covs_chol).permute(0, 2, 1)
        log_det = torch.logdet(precision_col)
        diffs = X[:, None, :] - self.means[None, :, :]
        log_prob = torch.einsum('nkd,kdj->nkj', diffs,  precision_col)
        log_prob = (log_prob ** 2).sum(dim=2)
        log_prob = -0.5 * (D * torch.log(2 * torch.Tensor([torch.pi])) + log_prob) + log_det
        return log_prob
    
    def maximize(self, X, gamma, xi):
        self.initial_prob = gamma[0] / gamma[0].sum()
        self.transition_prob = xi.sum(dim=0)/ xi.sum(dim=0).sum(dim=1, keepdims=True)
        
        self.means = torch.matmul(gamma.T, X) / gamma.sum(dim=0)[:, None]
        diffs = X[:, None, :] - self.means[None, :, :]
        covs = torch.einsum('nkdj,nkjt->nkdt', diffs.unsqueeze(3), diffs.unsqueeze(2))  # (N, K, D, D)
        self.covs = (covs * gamma[:, :, None, None]).sum(dim=0) / gamma.sum(dim=0)[:, None, None]

In [4]:
def draw(n, K, means, covs, initial_prob, transition_prob):
    dists = []
    for k in range(K):
        dists.append(torch.distributions.MultivariateNormal(means[k], covs[k]))

    seq = []
    hid = np.random.choice(K, p=initial_prob.numpy())
    for _ in range(n):
        seq.append(dists[hid].sample())
        hid = np.random.choice(K, p=transition_prob[hid].numpy())
    return torch.stack(seq)

In [23]:
n = 200
K = 3
means = torch.Tensor([[0, 0], [4, 7], [10, 2]])
covs = torch.Tensor([
    [[1, 0.01],
     [0.01, 1]],
    [[1.5, -0.1],
     [-0.1, 1.5]],
    [[1.5, 1],
     [1, 1.5]]
])
pi = torch.rand(K)
pi = pi / pi.sum()
A = torch.rand(K, K)
A = A / A.sum(dim=1, keepdims=True)

In [24]:
seq = draw(n, K, means, covs, pi, A)
seq = seq.double()

In [40]:
m = GaussianHMM(K=3, ndim=2)

In [ ]:
m.fit(seq, 100)

In [ ]:
gamma, xi, _ = m.expect(seq)
categories = gamma.argmax(dim=1)